In [1]:
import pandas as pd
import numpy as np
import random as rd
import datetime as dt
import matplotlib.pyplot as plt
import sys
%pylab inline

sys.path.append('D:/Python/AntoTradingSystem/gdax')

import gdax
import json

import myfunctions as myf

loc ='D:/Python/AntoTradingSystem/gdax/'

Populating the interactive namespace from numpy and matplotlib


C:\Users\Anto\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
import mygdax
newpk = mygdax.pk()
auth_client = gdax.AuthenticatedClient(newpk.key, newpk.b64secret, newpk.passphrase)

In [4]:
#auth_client.get_accounts()

In [ ]:
public_client = gdax.PublicClient()
public_client.get_product_order_book('BTC-USD', level=1)

x = public_client.get_product_ticker(product_id='BTC-EUR')
public_client.get_products()

pd.DataFrame(list(x.items()))

In [ ]:
public_client = gdax.PublicClient()
public_client.get_product_order_book('BTC-USD', level=2)
x = public_client.get_product_ticker(product_id='ETH-USD')
public_client.get_products()

pd.DataFrame(list(x.items()))

In [ ]:
x = public_client.get_product_order_book('BTC-EUR', level=3)
#pd.DataFrame(list(x.items()))
print(public_client.get_time())
#pd.DataFrame.from_dict(x).head(20)

def aggr_order_book3(x):
    typs = (('asks',True),('bids',False))
    df = pd.DataFrame()
    for typ in typs:
        pd.DataFrame(x[typ[0]])
        df_ = pd.DataFrame(x[typ[0]])[[0,1]]
        df_[0] = df_[0].map(lambda x: float(x))
        df_[1] = df_[1].map(lambda x: float(x))
        df_ = df_.groupby(0).sum()
        df_.columns = [typ[0]+'_size']
        df_[typ[0]+'_price'] = df_.index
        df_ = df_.sort_values(typ[0]+'_price', ascending=typ[1])
        df_ = df_.reset_index(drop=True)
        df_=df_[[typ[0]+'_price', typ[0]+'_size']]
        df = pd.concat([df,df_], axis=1)
    return df


In [ ]:
dfag = aggr_order_book3(x)
x_ = dfag['asks_size']
y_ = dfag['asks_price']
plt.scatter(x_, y_)

In [ ]:
dfag = aggr_order_book3(x)
x_ = dfag['bids_size']
y_ = dfag['bids_price']
plt.scatter(x_, y_)

In [ ]:
x = public_client.get_product_order_book('BTC-EUR', level=2)
#pd.DataFrame(list(x.items()))
print(public_client.get_time())
#pd.DataFrame.from_dict(x)

ask=pd.DataFrame(x['asks'], columns=['ask_price','ask_size','ask_num'])
bid=pd.DataFrame(x['bids'], columns=['bid_price','bid_size','bid_num'])

pd.concat([ask, bid], axis=1)

In [ ]:
y = public_client.get_product_trades(product_id='BTC-EUR')

pd.DataFrame.from_dict(y).head()


In [ ]:
z = public_client.get_product_historic_rates('BTC-EUR', 
                                             start='2017-11-10T10:30:00Z', 
                                             end='2017-11-10T12:30:00Z',
                                             granularity=20)

#z = public_client.get_product_historic_rates('BTC-EUR')
dfh = pd.DataFrame(z, columns=['time','low','high','open','close','volume'])
dfh['time'] = dfh['time'].map(lambda x: dt.datetime.utcfromtimestamp(x).isoformat())
dfh.index = dfh['time']
dfh = dfh.sort_values('time')
dfh[[1,2,3,4]].plot()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xticks(rotation=45)

In [ ]:
t = public_client.get_time()
dft = pd.DataFrame(list(t.items()))
dft